### Initial Setup

The following cells install dependencies and setup LLM tracing.


In [ ]:
%pip install -r requirements.txt

In [ ]:
# load required env vars for ddtrace
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import ddtrace.auto

ddtrace.patch_all()

from ddtrace.llmobs import LLMObs

LLMObs.enable()

### Tracing a Workflow Span

In this notebook, we are building a service that takes a free text query about art from a user, and feeds it into the Metropolitan Museum of Art API to get a list of artwork.

The steps are:

1. Take a query from a user
2. Parse that query via a call to OpenAI
3. Send the parsed query to the [Metropolitan Museum of Art API](https://metmuseum.github.io/#search)
4. Return a list of urls to the user


**1. Creating the tool to fetch data from the Met API**

In the next cell, we create and instrument a "tool" to send a query to the MET API's `/search` endpoint. The query will be created by a LLM call in a following cell.


In [ ]:
import requests
from ddtrace.llmobs.decorators import tool
import json

SEARCH_ENDPOINT = "https://collectionapi.metmuseum.org/public/collection/v1/search"
MAX_RESULTS = 5


@tool(name="fetch_met_urls")
def fetch_met_urls(query_parameters):
    LLMObs.annotate(
        input_data=json.dumps(query_parameters),
    )
    response = requests.get(SEARCH_ENDPOINT, params=query_parameters)
    response.raise_for_status()
    object_ids = response.json().get("objectIDs")
    objects_to_return = object_ids[:MAX_RESULTS] if object_ids else []
    urls = [
        f"https://www.metmuseum.org/art/collection/search/{objectId}"
        for objectId in objects_to_return
    ]
    LLMObs.annotate(
        output_data=json.dumps(urls),
    )
    return urls

**2. Creating an LLM call to handle parsing user input into a standardized query**


In [ ]:
from openai import OpenAI
import os
import json
from ddtrace.llmobs.decorators import workflow
from ddtrace.llmobs import LLMObs


oai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# we have scraped the Met API docs (https://metmuseum.github.io/#search)
# and saved the parameters in a json structure that we can inline into the prompt:
with open("met-search-endpoint-parameters.json", "r") as f:
    parameter_info = json.load(f)

    sys_prompt = f"""
You are a service that provides information about the art in the collection of the Metropolitan Museum of Art.
You take a free text query and parse it into a format that can be sent to the Met API's search endpoint.

Here is some information about the parameters accepted by the search endpoint:

{parameter_info}

The `hasImages` parameter must always be set to `true`.

Parse a query out from the user's message in a JSON format. Example:

Prompt: I'd like to see some highlights from the Met collection. Can you show me some paintings with flowers or other still lifes from around the 1800s?

Response: {{
    "q": "flowers still life",
    "hasImages": "true",
    "isHighlight": "true",
    "dateBegin": "1800",
    "dateEnd": "1899",
    "medium": "Paintings"
}}
 """


def parse_query(message, prompt=sys_prompt):
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": message},
    ]
    response = (
        oai_client.chat.completions.create(
            messages=messages,
            model="gpt-3.5-turbo",
            response_format={"type": "json_object"},
        )
        .choices[0]
        .message.content
    )
    query_parameters = json.loads(response)
    print(f"Parsed query parameters: {query_parameters}")
    return query_parameters

**3. Creating the `find_artworks` function**

Finally, we are creating a `find_artworks` function here that will tie the LLM call and tool call together. We annotate this as a workflow span:


In [ ]:
@workflow(name="find_artworks")
def find_artworks(question):
    LLMObs.annotate(
        input_data=question,
    )
    query = parse_query(question)
    urls = fetch_met_urls(query)
    LLMObs.annotate(
        output_data=json.dumps(urls),
    )
    return urls

Let's try it out:


In [ ]:
urls = find_artworks("I'd like to see some art from Ancient Greece.")
print("returned urls:", urls)